In [1]:
import cv2
import numpy as np
import pandas as pd 
import tifffile as tiff
from utils import *

# Import old knee video
old_knee_video = tiff.imread("data\\1 aging_00000221.tif") # 312 frames, 1024 x 1280 resolution

# Import Excel data
knee_angle_data = pd.read_excel("data\\Knee flexion degrees .xlsx", engine='openpyxl', sheet_name="aging-3")
# print(knee_angle_data.keys())

# Select columns and rename
flexion_degree = knee_angle_data.filter(['Frame Number', 'flexion degree']).rename(columns={'flexion degree': 'Degree'})
extension_degree = knee_angle_data.filter(['Frame Number.1', 'Extension degree']).rename(columns={'Frame Number.1': 'Frame Number', 'Extension degree': 'Degree'})

# Vertically concatenate data
knee_degree = pd.concat([flexion_degree, extension_degree], axis=0, ignore_index=True)

# Drop NaNs and cast frame numbers to int
knee_degree = knee_degree.dropna().astype({'Frame Number': np.uint16})

# Set first and last frame numbers 
first_frame = knee_degree['Frame Number'].iloc[0] # First frame 
last_frame = knee_degree['Frame Number'].iloc[-1] # Last frame
frame_num = first_frame # Initialize frame_num
total_frames = last_frame - first_frame

print(f"first frame = {first_frame}, last frame = {last_frame}")
print(f"total frames = {total_frames}")
frames_per_second = 60
print(f"frames_per_second = {frames_per_second}")
print(f"seconds/cycle = {total_frames/frames_per_second}")

# Iterate over all frames 
while frame_num <= last_frame:   
   
    # Get current frame
    frame = old_knee_video[frame_num, :, :]
    frame = np.rot90(frame, k = -1)

    # Preprocessing
    frame, translation_mx = centroid_stabilization(frame) # Center the frame
    frame = crop_image_with_margins(frame, 350,300,350,300)

    # Get dimensions
    h,w = frame.shape

    # Get binary mask
    _, binary_mask = cv2.threshold(frame, 0, 255, cv2.THRESH_OTSU)

    # Overlay mask
    # frame = np.maximum(frame, binary_mask)

    # Get rotation angle
    # degree = knee_degree.loc[knee_degree['Frame Number'] == frame_num, "Degree"].values[0]
    # degree = degree * 0.45
    # rotation_matrix = cv2.getRotationMatrix2D((w//2, h//2), degree, 1)
    # no_rotation = cv2.getRotationMatrix2D((0,0), 0, 2)

    # Apply rotation
    # rotated_frame = cv2.warpAffine(frame, rotation_matrix, (w,h))

    # Commit frame rotation
    # frame = rotated_frame

    # Rescale frame
    pixel_scale = 10.1119 # pixels / mm
    frame, rescale_factor = rescale_frame(frame, pixel_scale, scale_factor = 1.4)

    # Draw 1 cm scale bar along bottom of frame
    # draw_scale_bar(frame, pixel_scale*rescale_factor)

    # Print frame number on bottom-left corner
    h,w = frame.shape
    position = (10, h - 10)
    cv2.putText(frame, str(frame_num), position, 
                fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.7, 
                color = (255,255,255), thickness = 2, lineType = cv2.LINE_AA)

    # Write scale length on frame
    cv2.putText(frame, f"({pixel_scale*10} px = 1 cm)", (11*w//16, h-15), 
                fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.4, 
                color=(195,195,195), thickness=1, lineType=cv2.LINE_AA)

    # Display results
    # frame_combined = np.hstack((rotated_frame, frame))
    # cv2.imshow("Original Frame", frame)
    # cv2.imshow(f"Original vs. Rotated Frames ({knee_degree['Frame Number'].values[0]} to {knee_degree['Frame Number'].values[-1]})", frame_combined)    
    cv2.imshow(f"Aging knee: Frames {first_frame} to {last_frame}", frame)    

    # Increment frame number 
    frame_num += 1

    # Optional: Loop video
    loop = True
    if loop and frame_num > last_frame:
        frame_num = first_frame

    # Press 'q' to quit
    if cv2.waitKey(70) == ord('q'):
        break

cv2.destroyAllWindows()

first frame = 175, last frame = 222
total frames = 47
frames_per_second = 60
seconds/cycle = 0.7833333333333333


# Update for 10 Feb 2025:
- looked at the old knee video and loaded the angle data 
- cleaned up some of the previous code and wrote it as reusable functions
- tried to apply the angle data but it doesn't really work as expected

# Update for 24 Feb 2025 
- implemented scale length 

# TODO: 
- put the videos side by side to compare young vs. old knee fluid flow 
- goal of researchers is to see if there is a way to classify the young vs. old knee by observing the knee video

# TODO (24 Feb)
- wait for 4 points to have intensity diagram for fluid flow for the aging knee
- get the rough location of the femur by drawing a line between the center and the left/right lowest points 
- rotate the image so the femur is horizontal 
- try to draw the line for the femur and lower leg (requires angle data)

- prepare presentation on: rescaled aging knee/normal knee video -> comparison
- approximations for femur and lower leg, if possible

Spatial scale: 10.1119 pixels/mm
Time scale: 60 fps

# Notes (28 Feb):
- knee angle was measured between the femur and tibia
- knee angle data was measured by taking the average angle movement between flexion and extension

# TODO (28 Feb):
- put the videos side by side, time-stretch them so the cycles are aligned for easier analysis 
- try a similar concept to how I rescaled the frames using the pixel_scale

Experimenting with adaptive thresholding

In [9]:
import cv2
import numpy as np
import pandas as pd 
import tifffile as tiff
from utils import *

# Import old knee video
old_knee_video = tiff.imread("data\\1 aging_00000221.tif") # 312 frames, 1024 x 1280 resolution

# Import Excel data
data_set = "aging-1" # Change sheet using this parameter 
knee_angle_data = pd.read_excel("data\\Knee flexion degrees .xlsx", engine='openpyxl', sheet_name = data_set) 
# print(knee_angle_data.keys())

# "For Dr. Yu"
# # Video output settings 
# output_video_path = "output\\femur_estimation_demo_60_fps.avi"
# frame_width = 450*3
# frame_height = 450
# fps = 60
# output = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'DIVX'), fps, (frame_width, frame_height))
# "For Dr. Yu"

# Select columns and rename
flexion_degree = knee_angle_data.filter(['Frame Number', 'flexion degree']).rename(columns={'flexion degree': 'Degree'})
extension_degree = knee_angle_data.filter(['Frame Number.1', 'Extension degree']).rename(columns={'Frame Number.1': 'Frame Number', 'Extension degree': 'Degree'})

# Vertically concatenate data
knee_degree = pd.concat([flexion_degree, extension_degree], axis=0, ignore_index=True)

# Drop NaNs and cast frame numbers to int
knee_degree = knee_degree.dropna().astype({'Frame Number': np.uint16})

# Set first and last frame numbers 
first_frame = knee_degree['Frame Number'].iloc[0] # First frame 
last_frame = knee_degree['Frame Number'].iloc[-1] # Last frame
frame_num = first_frame # Initialize frame_num
total_frames = last_frame - first_frame

print(f"first frame = {first_frame}, last frame = {last_frame}")
print(f"total frames = {total_frames}")
frames_per_second = 60
print(f"frames_per_second = {frames_per_second}")
print(f"seconds/cycle = {total_frames/frames_per_second}")

# Iterate over all frames
loop_repeats = 0
while frame_num <= last_frame:   
   
    "Frame pre-processing"

    # Get current frame
    frame = old_knee_video[frame_num, :, :]
    frame = np.rot90(frame, k = -1)

    # Preprocessing
    frame, translation_mx = centroid_stabilization(frame, blur_strength=2) 
    frame = crop_square_frame(frame, n = 450) 
    frame_blurred = cv2.GaussianBlur(frame, (31,31), 0)

    # Save original frame
    frame_original = frame.copy()

    # Get dimensions
    h,w = frame.shape

    "Segmentation and boundary drawing"

    # Get adaptive mean mask (better)
    mean_mask = cv2.adaptiveThreshold(frame_blurred,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,71, -2)
    
    # Get adaptive gaussian mask
    gaussian_mask = cv2.adaptiveThreshold(frame_blurred,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,71, -2)
    
    # Clean up adaptive mean mask
    kernel_size = (15,15)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size)
    mean_mask = cv2.morphologyEx(mean_mask, cv2.MORPH_OPEN, kernel)

    # Get contours of mean mask
    _, mean_contours, _ = cv2.findContours(mean_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours on frame
    # cv2.drawContours(frame, mean_contours, -1, (0,0,255), 1)

    # Show mean mask
    # cv2.imshow("mean mask", mean_mask)

    # TEMPORARY
    # frame = mean_mask

    "Estimating the femur"

    # Define vertical midpoint for the frame
    vertical_cutoff = 0.45
    vertical_midpoint = int(h*vertical_cutoff)

    # Split mean mask into top and bottom parts
    if frame_num in range(51, 67): # TODO: this could be cleaned up and generalized 
        bottom_slice = vertical_midpoint + 40 # Set custom width of the bottom mask
    else: 
        bottom_slice = h
    top_mask = mean_mask[:vertical_midpoint, :]
    bottom_mask = mean_mask[vertical_midpoint:bottom_slice, :]

    # Show the top and bottom masks
    # cv2.imshow("bottom_mask", bottom_mask)
    # cv2.imshow("top_mask", top_mask)

    # Find closest points to the left of the frame
    top_left_point = find_closest_point(top_mask, axis = 1)
    bottom_left_point = find_closest_point(bottom_mask, axis = 1)
    bottom_left_point = (bottom_left_point[0],
                         bottom_left_point[1] + vertical_midpoint) # adjust the coordinate for plotting on the original frame

    # Get the midpoint between the left points
    left_midpoint = ( (int(round(top_left_point[0] + bottom_left_point[0]) / 2)), 
                      (int(round(top_left_point[1] + bottom_left_point[1]) / 2)) )
    
    # Draw the points
    # cv2.circle(frame, top_left_point, 3, (255,255,255), -1)
    # cv2.circle(frame, bottom_left_point, 3, (255,255,255), -1)
    # cv2.circle(frame, left_midpoint, 3, (255,255,255), -1)

    # Draw a point in the center of the frame
    centre_point = (round(w/2), int(h*0.495))
    cv2.circle(frame, centre_point, 3, (255,255,255), -1)

    # Draw the line representing the femur
    # cv2.line(frame, centre_point, left_midpoint, (255,255,255), 2)

    # Compute vector from centre_point to left_midpoint
    delta_x = left_midpoint[0] - centre_point[0]
    delta_y = left_midpoint[1] - centre_point[1]

    # Compute new endpoint by extending the vector
    new_endpoint = (centre_point[0] + 2 * delta_x, centre_point[1] + 2 * delta_y)

    # Draw the extended line
    # cv2.line(frame, centre_point, new_endpoint, (255,255,255), 2)

    # Get the angle drawn between the line and the horizontal axis 
    delta = [ 
        centre_point[0] - left_midpoint[0], # (delta_x, delta_y)
        centre_point[1] - left_midpoint[1]
        ] 
    angle = np.arctan2(delta[1], delta[0])
    angle_deg = np.degrees(angle)

    # Rotate the frame by the angle
    h,w = frame.shape
    # rotation_matrix = cv2.getRotationMatrix2D((round(w / 2), round(h / 2)), angle_deg, 1.0)
    # frame = cv2.warpAffine(frame, rotation_matrix, (w,h)) # Commit rotation



    "Displaying the frames"

    # Overlay mask
    # frame = np.maximum(frame, mean_mask)
    # frame_gaussian = np.maximum(frame, gaussian_mask)

    # Rescale frame
    pixel_scale = 10.1119 # pixels / mm
    frame, rescale_factor = rescale_frame(frame, pixel_scale, scale_factor = 1.5)

    # Draw reference lines
    # draw_reference_lines(frame, horizontal_line_placement = 0.495)

    # Print current frame number on the bottom-left corner
    h,w = frame.shape
    bottom_left_position = (10, h - 10)
    cv2.putText(frame, str(frame_num), bottom_left_position, 
                fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.7, 
                color = (255, 255, 255), thickness = 1, lineType=cv2.LINE_AA)
    
    # Print frame information on the top-left corner
    top_left_position = (10, 20)
    cv2.putText(frame_original, "Original frame (centered and cropped)", top_left_position, 
                fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.5, 
                color = (255, 255, 255), thickness = 1, lineType=cv2.LINE_AA)
    cv2.putText(mean_mask, "Adaptive mean thresholding", top_left_position, 
                fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.5, 
                color = (255, 255, 255), thickness = 1, lineType=cv2.LINE_AA)
    cv2.putText(frame, "Estimating the femur position", top_left_position, 
                fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.5, 
                color = (255, 255, 255), thickness = 1, lineType=cv2.LINE_AA)
    
    # Draw 1 cm scale along the bottom of the frame
    draw_scale_bar(frame, pixel_scale, rescale_factor) # Rescale the drawn bar 

    # Write scale length on frame
    cv2.putText(frame, f"({pixel_scale*10} px = 1 cm)", (21*w//32, h-15), 
                fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.4, 
                color=(195,195,195), thickness=1, lineType=cv2.LINE_AA)

    # Display results
    # combined_frame, combined_frame_shape = display_side_by_side([frame_original, mean_mask, frame])
    # frame_out = combined_frame
    frame_out = frame
    cv2.imshow(f"Aging knee: Frames {first_frame} to {last_frame}", frame_out)    

    "Miscellaneous technical things "

    # Write to file
    # write_to_file = False
    # if write_to_file:
    #     # print(f"writing frame {frame_num}! dimensions {frame_out.shape}")
    #     frame_out = cv2.cvtColor(frame_out, cv2.COLOR_GRAY2BGR)
    #     output.write(frame_out)

    # Increment frame number 
    frame_num += 1

    # Optional: Loop video
    loop = True
    if loop and frame_num > last_frame:
        loop_repeats += 1
        frame_num = first_frame

    # Increment loop repeats
    max_loop_repeats = -1 # -1 to stop
    if loop_repeats == max_loop_repeats:
        break

    # Press 'q' to quit
    if cv2.waitKey(70) == ord('q'):
        break

# output.release()
cv2.destroyAllWindows()
print("program done!")

first frame = 38, last frame = 81
total frames = 43
frames_per_second = 60
seconds/cycle = 0.7166666666666667
program done!


# TODO: (Mar 4) (Update: Mar 10)
- use Huizhu's new coordinates to estimate the fluid movement (aging knee) through the three parts using the change in intensity values (do this first)
- get the total intensity in each part 
- plot total intensity in each part as a function of time (frame number)
- ask Juan for his code to see how he did it 

- try the femur location estimation on the normal knee video (refine this later)
- try to divide the knee into N segments in a 360 degree circle (plot)
- try to estimate the tibia location using the knee angle data 
- try to get the line representing the meniscus (very hard? maybe Darcy can try this)

- try to time-align the two videos (for visualization purposes)

- ~~ give Dr. Yu a side-by-side video of: (left side), original video centered and cropped; (middle side) centered original video with contour lines, femur line; (right side) binary mask ~~

# Notes for next meeting with Dr. Chen:
- we want to demonstrate we can make whatever tools we want, so we can ask for new data 
- address concerns about the experimental conditions being different between the normal and aging knees (not same resolution, different flexion/extension cycle durations)

Variant pre-rotating the frame

In [13]:
import cv2
import numpy as np
import pandas as pd 
import tifffile as tiff
from utils import *

# Import old knee video
old_knee_video = tiff.imread("data\\1 aging_00000221.tif") # 312 frames, 1024 x 1280 resolution

# Import Excel data
data_set = "aging-1" # Change sheet using this parameter 
knee_angle_data = pd.read_excel("data\\Knee flexion degrees .xlsx", engine='openpyxl', sheet_name = data_set) 
# print(knee_angle_data.keys())

# "For Dr. Yu"
# # Video output settings 
# output_video_path = "output\\femur_estimation_demo_60_fps.avi"
# frame_width = 450*3
# frame_height = 450
# fps = 60
# output = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'DIVX'), fps, (frame_width, frame_height))
# "For Dr. Yu"

# Select columns and rename
flexion_degree = knee_angle_data.filter(['Frame Number', 'flexion degree']).rename(columns={'flexion degree': 'Degree'})
extension_degree = knee_angle_data.filter(['Frame Number.1', 'Extension degree']).rename(columns={'Frame Number.1': 'Frame Number', 'Extension degree': 'Degree'})

# Vertically concatenate data
knee_degree = pd.concat([flexion_degree, extension_degree], axis=0, ignore_index=True)

# Drop NaNs and cast frame numbers to int
knee_degree = knee_degree.dropna().astype({'Frame Number': np.uint16})

# Set first and last frame numbers 
first_frame = knee_degree['Frame Number'].iloc[0] # First frame 
last_frame = knee_degree['Frame Number'].iloc[-1] # Last frame
frame_num = first_frame # Initialize frame_num
total_frames = last_frame - first_frame

print(f"first frame = {first_frame}, last frame = {last_frame}")
print(f"total frames = {total_frames}")
frames_per_second = 60
print(f"frames_per_second = {frames_per_second}")
print(f"seconds/cycle = {total_frames/frames_per_second}")

# Iterate over all frames
loop_repeats = 0
while frame_num <= last_frame:   
   
    "Frame pre-processing"

    # Get current frame
    frame = old_knee_video[frame_num, :, :]
    frame = np.rot90(frame, k = -1)

    # Preprocessing
    frame, translation_mx = centroid_stabilization(frame, blur_strength=2) 
    frame = crop_square_frame(frame, n = 1000) 
    h,w = frame.shape

    # Pre-rotate the frame
    angle = 20
    rotation_matrix = cv2.getRotationMatrix2D((round(w / 2), round(h / 2)), angle, 1.0)
    frame = cv2.warpAffine(frame, rotation_matrix, (w,h)) # Commit rotation

    # Crop the frame
    frame = crop_square_frame(frame, n = 450) 
    h,w = frame.shape

    # Get blurred frame for simpler processing
    frame_blurred = cv2.GaussianBlur(frame, (31,31), 0)

    # Save original frame
    frame_original = frame.copy()
    


    "Segmentation and boundary drawing"

    # Get adaptive mean mask (better)
    mean_mask = cv2.adaptiveThreshold(frame_blurred,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,71, -2)
    
    # Get adaptive gaussian mask
    gaussian_mask = cv2.adaptiveThreshold(frame_blurred,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,71, -2)
    
    # Clean up adaptive mean mask
    kernel_size = (15,15)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size)
    mean_mask = cv2.morphologyEx(mean_mask, cv2.MORPH_OPEN, kernel)

    # Get contours of mean mask
    _, mean_contours, _ = cv2.findContours(mean_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours on frame
    cv2.drawContours(frame, mean_contours, -1, (0,0,255), 1)

    # Show mean mask
    # cv2.imshow("mean mask", mean_mask)

    # TEMPORARY
    # frame = mean_mask

    "Estimating the femur"

    # Define vertical midpoint for the frame
    vertical_cutoff = 0.50
    vertical_midpoint = int(h*vertical_cutoff)

    # Split mean mask into top and bottom parts
    if frame_num in range(51, 67): # TODO: this could be cleaned up and generalized 
        bottom_slice = vertical_midpoint + 40 # Set custom width of the bottom mask
    else: 
        bottom_slice = h
    bottom_slice = h
    top_mask = mean_mask[:vertical_midpoint, :]
    bottom_mask = mean_mask[vertical_midpoint:bottom_slice, :]

    # Show the top and bottom masks
    # cv2.imshow("bottom_mask", bottom_mask)
    # cv2.imshow("top_mask", top_mask)

    # Find closest points to the left of the frame
    top_left_point = find_closest_point(top_mask, axis = 1)
    bottom_left_point = find_closest_point(bottom_mask, axis = 1)
    bottom_left_point = (bottom_left_point[0],
                         bottom_left_point[1] + vertical_midpoint) # adjust the coordinate for plotting on the original frame

    # Get the midpoint between the left points
    left_midpoint = ( (int(round(top_left_point[0] + bottom_left_point[0]) / 2)), 
                      (int(round(top_left_point[1] + bottom_left_point[1]) / 2)) )
    
    # Draw the points
    cv2.circle(frame, top_left_point, 3, (255,255,255), -1)
    cv2.circle(frame, bottom_left_point, 3, (255,255,255), -1)
    cv2.circle(frame, left_midpoint, 3, (255,255,255), -1)

    # Draw a point in the center of the frame
    centre_point = (round(w/2), int(h*0.495))
    cv2.circle(frame, centre_point, 3, (255,255,255), -1)

    # Draw the line representing the femur
    # cv2.line(frame, centre_point, left_midpoint, (255,255,255), 2)

    # Compute vector from centre_point to left_midpoint
    delta_x = left_midpoint[0] - centre_point[0]
    delta_y = left_midpoint[1] - centre_point[1]

    # Compute new endpoint by extending the vector
    new_endpoint = (centre_point[0] + 2 * delta_x, centre_point[1] + 2 * delta_y)

    # Draw the extended line
    cv2.line(frame, centre_point, new_endpoint, (255,255,255), 2)

    # Get the angle drawn between the line and the horizontal axis 
    delta = [ 
        centre_point[0] - left_midpoint[0], # (delta_x, delta_y)
        centre_point[1] - left_midpoint[1]
        ] 
    angle = np.arctan2(delta[1], delta[0])
    angle_deg = np.degrees(angle)

    # Rotate the frame by the angle
    h,w = frame.shape
    rotation_matrix = cv2.getRotationMatrix2D((round(w / 2), round(h / 2)), angle_deg, 1.0)
    frame = cv2.warpAffine(frame, rotation_matrix, (w,h)) # Commit rotation



    "Displaying the frames"

    # Overlay mask
    # frame = np.maximum(frame, mean_mask)
    # frame_gaussian = np.maximum(frame, gaussian_mask)

    # Rescale frame
    pixel_scale = 10.1119 # pixels / mm
    frame, rescale_factor = rescale_frame(frame, pixel_scale, scale_factor = 1.4)

    # Draw reference lines
    # draw_reference_lines(frame, horizontal_line_placement = 0.495)

    # Print current frame number on the bottom-left corner
    h,w = frame.shape
    bottom_left_position = (10, h - 10)
    cv2.putText(frame, str(frame_num), bottom_left_position, 
                fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.7, 
                color = (255, 255, 255), thickness = 1, lineType=cv2.LINE_AA)
    
    # Print frame information on the top-left corner
    top_left_position = (10, 20)
    # cv2.putText(frame_original, "Original frame (centered and cropped)", top_left_position, 
    #             fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.5, 
    #             color = (255, 255, 255), thickness = 1, lineType=cv2.LINE_AA)
    # cv2.putText(mean_mask, "Adaptive mean thresholding", top_left_position, 
    #             fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.5, 
    #             color = (255, 255, 255), thickness = 1, lineType=cv2.LINE_AA)
    # cv2.putText(frame, "Estimating the femur position", top_left_position, 
    #             fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.5, 
    #             color = (255, 255, 255), thickness = 1, lineType=cv2.LINE_AA)
    
    # Draw 1 cm scale along the bottom of the frame
    draw_scale_bar(frame, pixel_scale, rescale_factor) # Rescale the drawn bar 

    # Write scale length on frame
    cv2.putText(frame, f"({pixel_scale*10} px = 1 cm)", (21*w//32, h-15), 
                fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.4, 
                color=(195,195,195), thickness=1, lineType=cv2.LINE_AA)

    # Display results
    # combined_frame, combined_frame_shape = display_side_by_side([frame_original, mean_mask, frame])
    # frame_out = combined_frame
    frame_out = frame
    cv2.imshow(f"Aging knee: Frames {first_frame} to {last_frame}", frame_out)    

    "Miscellaneous technical things "

    # Write to file
    # write_to_file = False
    # if write_to_file:
    #     # print(f"writing frame {frame_num}! dimensions {frame_out.shape}")
    #     frame_out = cv2.cvtColor(frame_out, cv2.COLOR_GRAY2BGR)
    #     output.write(frame_out)

    # Increment frame number 
    frame_num += 1

    # Optional: Loop video
    loop = True
    if loop and frame_num > last_frame:
        loop_repeats += 1
        frame_num = first_frame

    # Increment loop repeats
    max_loop_repeats = -1 # -1 to stop
    if loop_repeats == max_loop_repeats:
        break

    # Press 'q' to quit
    if cv2.waitKey(70) == ord('q'):
        break

# output.release()
cv2.destroyAllWindows()
print("program done!")

first frame = 38, last frame = 81
total frames = 43
frames_per_second = 60
seconds/cycle = 0.7166666666666667
program done!


Experiment to center around the femur (no results)

In [14]:
import cv2
import numpy as np
import pandas as pd 
import tifffile as tiff
from utils import *

# Import old knee video
old_knee_video = tiff.imread("data\\1 aging_00000221.tif") # 312 frames, 1024 x 1280 resolution

# Import Excel data
data_set = "aging-1" # Change sheet using this parameter 
knee_angle_data = pd.read_excel("data\\Knee flexion degrees .xlsx", engine='openpyxl', sheet_name = data_set) 
# print(knee_angle_data.keys())

# Select columns and rename
flexion_degree = knee_angle_data.filter(['Frame Number', 'flexion degree']).rename(columns={'flexion degree': 'Degree'})
extension_degree = knee_angle_data.filter(['Frame Number.1', 'Extension degree']).rename(columns={'Frame Number.1': 'Frame Number', 'Extension degree': 'Degree'})

# Vertically concatenate data
knee_degree = pd.concat([flexion_degree, extension_degree], axis=0, ignore_index=True)

# Drop NaNs and cast frame numbers to int
knee_degree = knee_degree.dropna().astype({'Frame Number': np.uint16})

# Set first and last frame numbers 
first_frame = knee_degree['Frame Number'].iloc[0] # First frame 
last_frame = knee_degree['Frame Number'].iloc[-1] # Last frame
frame_num = first_frame # Initialize frame_num
total_frames = last_frame - first_frame

print(f"first frame = {first_frame}, last frame = {last_frame}")
print(f"total frames = {total_frames}")
frames_per_second = 60
print(f"frames_per_second = {frames_per_second}")
print(f"seconds/cycle = {total_frames/frames_per_second}")

# Iterate over all frames 
while frame_num <= last_frame:   
   
    # Get current frame
    frame = old_knee_video[frame_num, :, :]
    frame = np.rot90(frame, k = -1)

    # Preprocessing
    frame, translation_mx = centroid_stabilization(frame) 
    frame = crop_image_with_margins(frame, 400,250,400,250)
    frame_blurred = cv2.GaussianBlur(frame, (51,51), 0)

    # Get dimensions
    h,w = frame.shape

    # Get tozero mask
    _, tozero_mask = cv2.threshold(frame_blurred, 41, 255, cv2.THRESH_TOZERO)

    # Get background 

    # Invert tozero mask
    tozero_mask = 255 - tozero_mask + 1

    # Overlay mask
    # frame = np.maximum(frame, binary_mask)

    # Rescale frame
    pixel_scale = 10.1119 # pixels / mm
    # frame, rescale_factor = rescale_frame(frame, pixel_scale, scale_factor = 1.4)

    # Draw 1 cm scale bar along bottom of frame
    # draw_scale_bar(frame, pixel_scale*rescale_factor)

    # Draw reference lines
    draw_reference_lines(frame)

    # Print frame number on bottom-left corner
    h,w = frame.shape
    position = (10, h - 10)
    cv2.putText(frame, str(frame_num), position, 
                fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.7, 
                color = (255,255,255), thickness = 2, lineType = cv2.LINE_AA)

    # Write scale length on frame
    cv2.putText(frame, f"({pixel_scale*10} px = 1 cm)", (11*w//16, h-15), 
                fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.4, 
                color=(195,195,195), thickness=1, lineType=cv2.LINE_AA)
    
    # Invert frame
    # frame = 255 - frame

    # Display results
    frame_out = np.hstack([frame, tozero_mask])
    cv2.imshow(f"Aging knee: Frames {first_frame} to {last_frame}", frame_out)    

    # Increment frame number 
    frame_num += 1

    # Optional: Loop video
    loop = True
    if loop and frame_num > last_frame:
        frame_num = first_frame

    # Press 'q' to quit
    if cv2.waitKey(70) == ord('q'):
        break

cv2.destroyAllWindows()

first frame = 38, last frame = 81
total frames = 43
frames_per_second = 60
seconds/cycle = 0.7166666666666667
